In [1]:
import fitz
import re
import pytesseract
import cv2
import os
import time
# настройки для модуля распознавания текста

pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
tessdata_dir_config = r'--tessdata-dir "C:\\Program Files\\Tesseract-OCR\\tessdata"'

In [2]:
# получаем доступ к файлам

pdf_list = []
path = 'C:\\Users\\P\\Start folder for Jupyter\\PSB\\dataset\\Промсвязьбанк Датасет\\Тестовый dataset'
for root, dirs, files in os.walk(path):
    if len(files) > 0:
        for file in files:
            if file.endswith('.pdf'):
                pdf_list.append(root + '\\' + file)
# pdf_list

['C:\\Users\\P\\Start folder for Jupyter\\PSB\\dataset\\Промсвязьбанк Датасет\\Тестовый dataset\\ПАО НКХП 2315014748\\Описание бизнеса\\Презентация компании 2.pdf',
 'C:\\Users\\P\\Start folder for Jupyter\\PSB\\dataset\\Промсвязьбанк Датасет\\Тестовый dataset\\ПАО НКХП 2315014748\\Описание бизнеса\\Презентация компании.pdf',
 'C:\\Users\\P\\Start folder for Jupyter\\PSB\\dataset\\Промсвязьбанк Датасет\\Тестовый dataset\\ПАО НКХП 2315014748\\Финансовое досье\\2020\\4 квартал\\Бухгалтерская отчетность\\Аудиторское заключение.pdf',
 'C:\\Users\\P\\Start folder for Jupyter\\PSB\\dataset\\Промсвязьбанк Датасет\\Тестовый dataset\\ПАО НКХП 2315014748\\Финансовое досье\\2020\\4 квартал\\Бухгалтерская отчетность\\Форма 1.pdf',
 'C:\\Users\\P\\Start folder for Jupyter\\PSB\\dataset\\Промсвязьбанк Датасет\\Тестовый dataset\\ПАО НКХП 2315014748\\Финансовое досье\\2020\\4 квартал\\Бухгалтерская отчетность\\Форма 2.pdf',
 'C:\\Users\\P\\Start folder for Jupyter\\PSB\\dataset\\Промсвязьбанк Датасет\

In [3]:
os.chdir('C:\\Users\\P\\Start folder for Jupyter\\train_for_pdf')
os.getcwd()

'C:\\Users\\P\\Start folder for Jupyter\\train_for_pdf'

In [4]:
%%time

path_to_dir_with_img = []

for number, pdf_file in enumerate(pdf_list):
    path_from_list = pdf_file
    pdf_document = fitz.open(path_from_list)  
    start_dir = 'C:\\Users\\P\\Start folder for Jupyter\\train_for_pdf'
    os.chdir(start_dir)
    os.mkdir(str(number+1))
    path = start_dir + '\\'+ str(number+1)
    path_to_dir_with_img.append(path)
    
    for current_page in range(len(pdf_document)): 
        os.chdir(path)
        for image in pdf_document.getPageImageList(current_page):
            xref = image[0]
            pix = fitz.Pixmap(pdf_document, xref)
            if pix.n < 5:        # this is GRAY or RGB
                pix.writePNG("page%s-%s.png" % (current_page, xref))
            else:                # CMYK: convert to RGB first
                pix1 = fitz.Pixmap(fitz.csRGB, pix)
                pix1.writePNG("page%s-%s.png" % (current_page, xref))
                pix1 = None
            pix = None

Wall time: 57.6 s


In [5]:
path_to_dir_with_img

['C:\\Users\\P\\Start folder for Jupyter\\train_for_pdf\\1',
 'C:\\Users\\P\\Start folder for Jupyter\\train_for_pdf\\2',
 'C:\\Users\\P\\Start folder for Jupyter\\train_for_pdf\\3',
 'C:\\Users\\P\\Start folder for Jupyter\\train_for_pdf\\4',
 'C:\\Users\\P\\Start folder for Jupyter\\train_for_pdf\\5',
 'C:\\Users\\P\\Start folder for Jupyter\\train_for_pdf\\6',
 'C:\\Users\\P\\Start folder for Jupyter\\train_for_pdf\\7',
 'C:\\Users\\P\\Start folder for Jupyter\\train_for_pdf\\8',
 'C:\\Users\\P\\Start folder for Jupyter\\train_for_pdf\\9',
 'C:\\Users\\P\\Start folder for Jupyter\\train_for_pdf\\10']

In [21]:
# Словарь из: 
# названия документа (метка класса) и его атрибутов (список ключевых слов по которым будет осуществлен поиск и классификация).
# Словарь можно будет обновлять из оболочки приложения:
# т.е. если появится новый класс (название документа для классификации) или новые ключевые слова их можно будет добавить сюда

names = {"Положение о СД": ['Положение о совете директоров', 'Председатель совета директоров', 
                            'Письменное мнение', 'Опросный лист', 'Уведомление о проведении Совета директоров'],
        'Устав_действующий': ['Устав' , 'Уставный капитал', 'Органы управления', 'Резервный фонд', 'Бюллетени'],
        'Бухгалтерская отчетность_форма 1': ['Бухгалтерский баланс', 'Форма по ОКУД 0710001', 
                                             'Дата', 'Актив', 'Пассив'],
        'Бухгалтерская отчетность_форма 2': ['Отчет о финансовых результатах', 'Форма по ОКУД 0710002',
                                             'Дата', 'Чистая прибыль', 'Налог на прибыль'],
        'Бухгалтерская отчетность_форма 1 _промежуточная': ['Бухгалтерский баланс', 'Форма по ОКУД 0710001',
                                                             'Дата', 'Актив', 'Пассив'],
        'Бухгалтерская отчетность_форма 2 _промежуточная': ['Отчет о финансовых результатах', 'Форма по ОКУД 0710002',
                                             'Дата', 'Чистая прибыль', 'Налог на прибыль'],
        'Аудиторское заключение': ['Аудиторское заключение', 'Сведения об аудируемом лице', 'Сведения об аудиторе',
                                   'Основание для выражения мнения', 'Ответственность аудитора'],
        'Описание_деятельности_ГК': ['Презентация компании', 'История компании', 'Обзор рынка', 'Обзор компании'],
        'Решение_назначение ЕИО': ['Протокол совета директоров', 'Дата составления протокола',
                                   'Избрание/назначение генерального директора', 'Итоги голосования', 'Принятое решение']}


In [4]:
# os.getcwd()
# os.chdir('C:\\Users\\P\\Start folder for Jupyter\\train_for_pdf\\10')

In [6]:
# достаем адреса к изображениям после обработки pdf файла

file_name = []
path = os.getcwd()
for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith('.png'):
            file_name.append(root + '\\' + file)
# file_name

['C:\\Users\\P\\Start folder for Jupyter\\train_for_pdf\\10\\page0-307.png',
 'C:\\Users\\P\\Start folder for Jupyter\\train_for_pdf\\10\\page0-309.png',
 'C:\\Users\\P\\Start folder for Jupyter\\train_for_pdf\\10\\page0-311.png',
 'C:\\Users\\P\\Start folder for Jupyter\\train_for_pdf\\10\\page1-7.png',
 'C:\\Users\\P\\Start folder for Jupyter\\train_for_pdf\\10\\page10-85.png',
 'C:\\Users\\P\\Start folder for Jupyter\\train_for_pdf\\10\\page11-93.png',
 'C:\\Users\\P\\Start folder for Jupyter\\train_for_pdf\\10\\page12-101.png',
 'C:\\Users\\P\\Start folder for Jupyter\\train_for_pdf\\10\\page13-109.png',
 'C:\\Users\\P\\Start folder for Jupyter\\train_for_pdf\\10\\page14-119.png',
 'C:\\Users\\P\\Start folder for Jupyter\\train_for_pdf\\10\\page14-121.png',
 'C:\\Users\\P\\Start folder for Jupyter\\train_for_pdf\\10\\page14-123.png',
 'C:\\Users\\P\\Start folder for Jupyter\\train_for_pdf\\10\\page15-131.png',
 'C:\\Users\\P\\Start folder for Jupyter\\train_for_pdf\\10\\page16-139.

In [25]:
# простой счетчик класса
freq_classes = {'Устав_действующий': 0,
                'Положение о СД': 0,
                'Бухгалтерская отчетность_форма 1': 0,
                'Бухгалтерская отчетность_форма 2': 0,
                'Бухгалтерская отчетность_форма 1 _промежуточная': 0,
                'Бухгалтерская отчетность_форма 2 _промежуточная': 0,
                'Аудиторское заключение': 0,
                'Описание_деятельности_ГК': 0,
                'Решение_назначение ЕИО': 0}

In [28]:
%%time


pages = file_name
for page in pages:
    try:
        img = cv2.imread(page)
        text_page = pytesseract.image_to_string(img, lang= 'rus', config=tessdata_dir_config)
        for key, values in names.items():
            for val in values:
                if re.findall(val.lower(), text_page.lower()):
                    freq_classes[key] += 1

    except:
        print('Не верифицированно, отправить в соответствующую папку')

answer = sorted(freq_classes, key=lambda x: freq_classes.get(x), reverse=True)[0]
print(f'Документ распознан, имя: {answer}')

Документ распознан, имя: Устав_действующий
Wall time: 3min 36s


In [29]:
freq_classes

{'Устав_действующий': 37,
 'Положение о СД': 4,
 'Бухгалтерская отчетность_форма 1': 14,
 'Бухгалтерская отчетность_форма 2': 9,
 'Бухгалтерская отчетность_форма 1 _промежуточная': 14,
 'Бухгалтерская отчетность_форма 2 _промежуточная': 9,
 'Аудиторское заключение': 1,
 'Описание_деятельности_ГК': 0,
 'Решение_назначение ЕИО': 2}